# 系列データ（シークエンス）
系列データとは特定の順序で並んでいて、互いに無関係ではないようなデータである。対して、時系列データとは各データに時間の次元が関連付けられている系列データである。
- 株価、録音データ等 => 時系列データ
- テキストデータ、DNA配列 => 系列データ

## シーケンスモデルのカテゴリ
入出力のデータが系列データかどうかで分類される。
| カテゴリ |入力|出力|例|
|----|----|----|----|
|多対一|シーケンス|not シーケンス|感情分析|
|一対多|not シーケンス|シーケンス|画像キャプショニング|
|多対多|シーケンス|シーケンス|同期モデル：動画分類<br>遅延モデル：言語翻訳|

## シーケンスモデルの構造
RNNの構造には一般に系列データをモデル化するために、再帰的な要素が含まれている。これは例えば、隠れ層が1つ前の時間刻みの情報を得る構造（リカレントエッジ）になっている。
再規特性をどう持つかによって分類できる。
- 隠れ層から隠れ層への再帰
- 出力層から隠れ層への再帰
- 出力層から出力層への再帰
Tensorflow keras API の `SinpleRNN` は出力層から出力層への再帰構造のモデルである。

In [1]:
import tensorflow as tf

2024-10-20 09:56:24.166102: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-20 09:56:24.367721: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-20 09:56:24.448314: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-20 09:56:24.468085: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-20 09:56:24.613094: I tensorflow/core/platform/cpu_feature_guar

In [18]:
tf.random.set_seed(1)
model = tf.keras.Sequential()
rnn_layer = tf.keras.layers.SimpleRNN(
    units=2, use_bias=True, 
    return_sequences=True,
    input_shape=(None, 5)) # batch_size , features
model.add(rnn_layer)
# rnn_layer.build(input_shape=(None, 5))
# model.compile(loss='mse', optimizer='adam')

# weights = model.get_weights()
w_xh, w_oo, b_h = model.get_weights()
# print(weights)
# w_xh, w_oo, b_h = rnn_layer.weights
print('W_xh shape:', w_xh.shape)
print('W_oo shape:', w_oo.shape)
print('b_h shape:', b_h.shape)

W_xh shape: (5, 2)
W_oo shape: (2, 2)
b_h shape: (2,)


Gemini 曰く
- 最新のKerasでは、buildメソッドを明示的に呼び出す必要はありません。モデルのコンパイル時に自動的に呼び出されます。
- 最新のKerasでは、input_shapeはモデルのコンパイル時に自動的に設定されることが一般的です。
- SimpleRNNレイヤーにおいて、シーケンスの長さをinput_shapeで厳密に指定する必要はありません。

In [19]:
x_seq = tf.convert_to_tensor(
    [[1.0]*5, [2.0]*5, [3.0]*5],
    dtype=tf.float32)


## output of SimepleRNN:
output = rnn_layer(tf.reshape(x_seq, shape=(1, 3, 5))) # batch_size, sequence length, features

## manually computing the output:
out_man = []
for t in range(len(x_seq)):
    xt = tf.reshape(x_seq[t], (1, 5))
    print('Time step {} =>'.format(t))
    print('   Input           :', xt.numpy())
    
    ht = tf.matmul(xt, w_xh) + b_h    
    print('   Hidden          :', ht.numpy())
    
    if t>0:
        prev_o = out_man[t-1]
    else:
        prev_o = tf.zeros(shape=(ht.shape))
        
    ot = ht + tf.matmul(prev_o, w_oo)
    ot = tf.math.tanh(ot)
    out_man.append(ot)
    print('   Output (manual) :', ot.numpy())
    print('   SimpleRNN output:'.format(t), output[0][t].numpy())
    print()

Time step 0 =>
   Input           : [[1. 1. 1. 1. 1.]]
   Hidden          : [[ 0.36025524 -0.39987028]]
   Output (manual) : [[ 0.3454388 -0.379838 ]]
   SimpleRNN output: [ 0.3454388 -0.379838 ]

Time step 1 =>
   Input           : [[2. 2. 2. 2. 2.]]
   Hidden          : [[ 0.7205105  -0.79974055]]
   Output (manual) : [[ 0.5282858  -0.29476205]]
   SimpleRNN output: [ 0.5282858  -0.29476205]

Time step 2 =>
   Input           : [[3. 3. 3. 3. 3.]]
   Hidden          : [[ 1.0807656 -1.1996108]]
   Output (manual) : [[ 0.80702984 -0.5340762 ]]
   SimpleRNN output: [ 0.80702984 -0.5340762 ]

